In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D, ELU
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.layers.core import Lambda

from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


### Generator

In [24]:
import os
import csv

reduced = False

if reduced == True:
    csv_filepath = 'data-udacity/driving_log_reduced.csv'
else:
    csv_filepath = 'data-udacity/driving_log.csv'
samples = []
with open(csv_filepath) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
def add_to_samples(csv_filepath, samples):
    with open(csv_filepath) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    return samples

samples = add_to_samples('data-recovery-annie/driving_log.csv', samples)

samples = add_to_samples('data-udacity-flipped/driving_csv_flipped_headoff.csv)

# samples = add_to_samples('data-extra-laps/driving_log.csv', samples)

# samples = add_to_samples('data-recovery/driving_log.csv', samples)

# samples = add_to_samples('data-mouse/driving_log.csv', samples)


"""
samples = add_to_samples('data-udacity/driving_log_18_bridge_reduced2.csv)
"""
samples = samples[1:]
print("Samples: ", len(samples))    
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.1)

Samples:  9384


In [3]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data-udacity/'+batch_sample[0]
                # name = './data-udacity/IMG/'+batch_sample[0].split('/')[-1]
                center_image = mpimg.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            
            # print("X_train: ", X_train)
            # print("y_train: ", y_train)
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

### Models

In [4]:
def resize(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 66, 200)

def resize_comma(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 40, 160)

def resize_comma_10_40(image):
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 40, 160)

def halve_size(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, imshape[0] // 2, imshape[1] // 2)

def resize_80_160(image):
    imshape = image.shape
    import tensorflow as tf  # This import is required here otherwise the model cannot be loaded in drive.py
    return tf.image.resize_images(image, 80, 160)

In [27]:
# Comma.ai model
# https://github.com/commaai/research/blob/master/train_steering_model.py

model = Sequential()

# Crop 50 pixels from the top of the image and 20 from the bottom
model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize_comma))

# model.add(Lambda(lambda x: x[:,:,:,0:1]))

model.add(Lambda(lambda x: (x/255.0) - 0.5))

model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())

model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))

model.add(Flatten())
# model.add(Dropout(.2))
model.add(ELU())

model.add(Dense(512))
# model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(1))

adam = Adam(lr=0.0001)

model.compile(optimizer=adam, loss="mse", metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_7 (Cropping2D)        (None, 65, 320, 3)    0           cropping2d_input_7[0][0]         
____________________________________________________________________________________________________
lambda_18 (Lambda)               (None, 40, 160, 3)    0           cropping2d_7[0][0]               
____________________________________________________________________________________________________
lambda_19 (Lambda)               (None, 40, 160, 3)    0           lambda_18[0][0]                  
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 10, 40, 16)    3088        lambda_19[0][0]                  
___________________________________________________________________________________________

Data preprocessing reference: [Geoff Breemer](https://carnd-forums.udacity.com/questions/36045049/answers/36047341)

In [20]:
# NVIDIA version
model = Sequential()

# Crop the images
model.add(Cropping2D(cropping=((60, 20), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

# Normalise data
# TODO: some people use /255.0 - 0.5. Why?
model.add(Lambda(lambda x: x/127.5 - 1.))


model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal', name='conv1'))
model.add(ELU())
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal', name='conv2'))
model.add(ELU())
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal', name='conv3'))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal', name='conv4'))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal', name='conv5'))
model.add(Flatten(name='flatten1'))
model.add(ELU())
model.add(Dense(1164, init='he_normal', name='dense1'))
model.add(ELU())
model.add(Dense(100, init='he_normal', name='dense2'))
model.add(ELU())
model.add(Dense(50, init='he_normal', name='dense3'))
model.add(ELU())
model.add(Dense(10, init='he_normal', name='dense4'))
model.add(ELU())
model.add(Dense(1, init='he_normal', name='dense5'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(optimizer=adam, loss='mse')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_6 (Cropping2D)        (None, 80, 320, 3)    0           cropping2d_input_6[0][0]         
____________________________________________________________________________________________________
lambda_15 (Lambda)               (None, 66, 200, 3)    0           cropping2d_6[0][0]               
____________________________________________________________________________________________________
lambda_16 (Lambda)               (None, 66, 200, 1)    0           lambda_15[0][0]                  
____________________________________________________________________________________________________
lambda_17 (Lambda)               (None, 66, 200, 1)    0           lambda_16[0][0]                  
___________________________________________________________________________________________

### Train model

In [28]:
# Train model
batch_size = 32
nb_epoch = 20
 
checkpointer = ModelCheckpoint(filepath="./tmp/comma-v3g.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1, save_best_only=False)
    
model.fit_generator(train_generator, 
                    samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), nb_epoch=nb_epoch,
                    callbacks=[checkpointer])

Epoch 1/20
8430/8445 [============================>.] - ETA: 0s - loss: 0.0122 - acc: 0.5765

/Users/jessica/anaconda/lib/python3.5/site-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: saving model to ./tmp/comma-v3g.00-0.01.hdf5
8462/8445 [==============================] - 44s - loss: 0.0122 - acc: 0.5760 - val_loss: 0.0127 - val_acc: 0.5527
Epoch 2/20
8462/8445 [==============================] - 42s - loss: 0.0111 - acc: 0.5768 - val_loss: 0.0129 - val_acc: 0.5469
Epoch 3/20
8462/8445 [==============================] - 38s - loss: 0.0107 - acc: 0.5773 - val_loss: 0.0125 - val_acc: 0.5448
Epoch 4/20
8462/8445 [==============================] - 36s - loss: 0.0100 - acc: 0.5750 - val_loss: 0.0123 - val_acc: 0.5475
Epoch 5/20
8462/8445 [==============================] - 36s - loss: 0.0097 - acc: 0.5781 - val_loss: 0.0118 - val_acc: 0.5432
Epoch 6/20
8462/8445 [==============================] - 35s - loss: 0.0095 - acc: 0.5747 - val_loss: 0.0123 - val_acc: 0.5327
Epoch 7/20
8448/8445 [==============================] - 34s - loss: 0.0094 - acc: 0.5747 - val_loss: 0.0114 - val_acc: 0.5454
Epoch 8/20
8462/8445 [==============================] - 34s - loss: 0.0

In [29]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model-comma-v3g.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


### Deprecated code

In [15]:
# NVIDIA End to End Learning Pipeline Model

model = Sequential()

# Crop the images
model.add(Cropping2D(cropping=((60, 20), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

# Normalise data
# TODO: some people use /255.0 - 0.5. Why?
model.add(Lambda(lambda x: x/127.5 - 1.))


# Conv layer 1, 5x5 kernel to 24@ (from 3@)
# TODO: What does the number of filters MEAN?
# Stride of 2x2
model.add(Convolution2D(24, 5, 5,
                        input_shape=(66,200))
         )

model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 2, 5x5 kernel to 36@
model.add(Convolution2D(36, 5, 5))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 3, 5x5 kernel to 48@
model.add(Convolution2D(48, 5, 5))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 4, 3x3 kernel to 64@
model.add(Convolution2D(64, 3, 3))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Conv layer 5, 3x3 kernel to 64@
model.add(Convolution2D(64, 3, 3))
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.1))

# Flatten
model.add(Flatten())

# Removed: fully connected layer 1, 1164 neurons

# Fc2, 100 neurons
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Fc3, 50 neurons
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Fc4, 10 neurons
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# Output
model.add(Dense(1, activation="tanh"))

# Compile model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001)
model.compile(loss='mean_squared_error',
              optimizer=adam,
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_3 (Cropping2D)        (None, 80, 320, 3)    0           cropping2d_input_3[0][0]         
____________________________________________________________________________________________________
lambda_6 (Lambda)                (None, 66, 200, 3)    0           cropping2d_3[0][0]               
____________________________________________________________________________________________________
lambda_7 (Lambda)                (None, 66, 200, 1)    0           lambda_6[0][0]                   
____________________________________________________________________________________________________
lambda_8 (Lambda)                (None, 66, 200, 1)    0           lambda_7[0][0]                   
___________________________________________________________________________________________